# dsummary
Algorithm that mimics `summary` in R on distributed datasets. Which includes:
* Check header labels
* Count the number of rows
* `Min`, `Max`, `Q1`, `Q2`, `Q3` and Average of `Int64` and `float64` columns
* Categories of `category` columns

Limits of the algorithm:
* The entire dataset needs at least 10 records. Else no statistical analysis should be performed
* `category` columns needs at least 2 different values
* Column names (from the input) need to match the dataset column names

## Summary

In [184]:
import pandas
import json

### input.txt
The input.txt is mounted by the docker-container, and contains input to the algorithm.

The input for this algorithm include the method name that is called in the docker-container `summarize` and a `dict` containing column names and dtypes. The allowed types are: `object`, `int64`, `float64`, `bool`, `datetime64`, `category`

In [185]:
input_ = {
    "method":"summarize", 
    "columns":{
        "patient_id": 'Int64',
        "age": 'Int64',
        "weight": 'float64', 
        "stage": 'category',
        "cat": 'category',
        "hot_encoded": 'Int64'
    }
}

### database.csv
The database csv-file is mounted in the docker-container.

In [186]:
%pwd

'C:\\Users\\FMa1805.36838\\Repositories\\dSummary'

In [187]:
dataframe = pandas.read_csv("./local/database.csv", sep=";",decimal=",", dtype=input_.get("columns"))
dataframe

,patient_id,age,weight,stage,cat,hot_encoded
0,1,41,73.2,IV,Q,1
1,2,37,65.9,I,Q,NaN
2,3,45,84.1,II,Q,0
3,4,47,83.1,II,Q,0
4,5,33,NaN,NaN,Q,1
5,6,34,NaN,NaN,Q,1
6,7,34,NaN,NaN,Q,1
7,8,34,NaN,NaN,Q,1
8,9,34,NaN,NaN,Q,1
9,10,34,NaN,NaN,Q,1


In [188]:
dataframe.dtypes

patient_id        Int64
age               Int64
weight          float64
stage          category
cat            category
hot_encoded       Int64
dtype: object

### algorithm.py

In [189]:
# retrieve column names from the dataset
columns_series = pandas.Series(data=input_.get("columns"))
column_names = list(columns_series.keys())

# compare column names from dataset to the input column names
column_names_correct = column_names == list(input_.get("columns").keys())
print(f"column_names_correct={column_names_correct}")

column_names_correct=True


In [190]:
# count the number of rows in the dataset
number_of_rows = len(dataframe)
print(f"number_of_rows={number_of_rows}")

number_of_rows=10


In [218]:
# compute the avarage of the numeric columns
columns = {}
numeric_colums = columns_series.loc[columns_series.isin(['Int64','float64'])]

for column_name in numeric_colums.keys():
    column_values = dataframe[column_name]
    q1, median, q3 = column_values.quantile([0.25,0.5,0.75]).values
    mean = column_values.mean()
    minimum = column_values.min()
    maximum = column_values.max()
    nan = column_values.isna().sum()
    total = column_values.sum()
    std = column_values.std()
    sq_dev_sum = (column_values-column_values.mean()).pow(2).sum()
    columns[column_name] = {
        "min": minimum,
        "q1": q1,
        "median": median,
        "mean": mean,
        "q3": q3,
        "max": maximum,
        "nan": int(nan),
        "sum": total,
        "sq_dev_sum": sq_dev_sum,
        "std": std
    }
pandas.DataFrame.from_dict(columns,orient='index')

,min,q1,median,mean,q3,max,nan,sum,sq_dev_sum,std
age,33.0,34.000,34.00,37.300000,40.00,47.0,0,373.0,240.100000,5.165054
hot_encoded,0.0,1.000,1.00,0.777778,1.00,1.0,1,7.0,1.555556,0.440959
patient_id,1.0,3.250,5.50,5.500000,7.75,10.0,0,55.0,82.500000,3.027650
weight,65.9,71.375,78.15,76.575000,83.35,84.1,6,306.3,224.547500,8.651541


In [219]:
dataframe["stage"].cat.categories

Index(['I', 'II', 'IV'], dtype='object')

In [220]:
dataframe["stage"].value_counts().to_dict()

{'II': 2, 'IV': 1, 'I': 1}

In [221]:
# return the categories in categorial columns
categoral_colums = columns_series.loc[columns_series.isin(['category'])]

for column_name in categoral_colums.keys():
    
    columns[column_name] = dataframe[column_name].value_counts().to_dict()
    
    display(pandas.Series(columns[column_name]))

II    2
IV    1
I     1
dtype: int64

Q    10
dtype: int64

In [222]:
output = {
    "column_names_correct": column_names_correct,
    "number_of_rows": number_of_rows,
    "statistics": columns
}
output

{'column_names_correct': True,
 'number_of_rows': 10,
 'statistics': {'patient_id': {'min': 1,
   'q1': 3.25,
   'median': 5.5,
   'mean': 5.5,
   'q3': 7.75,
   'max': 10,
   'nan': 0,
   'sum': 55,
   'sq_dev_sum': 82.5,
   'std': 3.0276503540974917},
  'age': {'min': 33,
   'q1': 34.0,
   'median': 34.0,
   'mean': 37.3,
   'q3': 40.0,
   'max': 47,
   'nan': 0,
   'sum': 373,
   'sq_dev_sum': 240.10000000000002,
   'std': 5.165053511608353},
  'weight': {'min': 65.9,
   'q1': 71.375,
   'median': 78.15,
   'mean': 76.575,
   'q3': 83.35,
   'max': 84.1,
   'nan': 6,
   'sum': 306.3,
   'sq_dev_sum': 224.5474999999997,
   'std': 8.651541288502678},
  'hot_encoded': {'min': 0,
   'q1': 1.0,
   'median': 1.0,
   'mean': 0.7777777777777778,
   'q3': 1.0,
   'max': 1,
   'nan': 1,
   'sum': 7,
   'sq_dev_sum': 1.5555555555555556,
   'std': 0.44095855184409843},
  'stage': {'II': 2, 'IV': 1, 'I': 1},
  'cat': {'Q': 10}}}

### output.txt

In [223]:
with open("./local/output.txt", "w") as fp:
    json.dump(output,fp)

## Master
The master container computes the statistics at each site and computes the global statistics. The `Q1`, `Median` and `Q3` cannot be computed in a privacy preserving way. Therefore only the global `Min`, `Max`, `Mean` and number of `NaN`'s is reported. In computing the `Mean` the NaN values are ignored.

Things to keep in mind:
* if one or more of the sites has incorrect headers (column_names) then you will recieve `header_columns_correct:false` error
* if one or more of the sites contain less then 10 records then you will recieve a warning message `warn > Dataset has less than 10 rows. Exiting.`

In [233]:
results = [output]

In [234]:
g_stats = {}

# check that all dataset reported their headers are correct
g_stats["column_names_correct"] = all(x["column_names_correct"] for x in results)

# count the total number of rows of all datasets
g_stats["number_of_rows"] = sum(x["number_of_rows"] for x in results)

# compute global statics for numeric columns
for header in numeric_colums.keys():

    n = g_stats["number_of_rows"]
    
    # extract the statistics for each column and all results
    stats = [ result["statistics"][header] for result in results ] 
    
    # compute globals
    g_min = min([x.get("min") for x in stats])
    g_max = max([x.get("max") for x in stats])
    g_nan = sum([x.get("nan") for x in stats])
    g_mean = sum([x.get("sum") for x in stats]) / (n-g_nan)
    g_std = (sum([x.get("sq_dev_sum") for x in stats]) / (n-1-g_nan))**(0.5)
    
    # estimate the median
    # see https://stats.stackexchange.com/questions/103919/estimate-median-from-mean-std-dev-and-or-range
    u_std = (((n-1)/n)**(0.5)) * g_std 
    g_median = [
        max([g_min, g_mean - u_std]),
        min([g_max, g_mean + u_std])
    ]
    
    g_stats[header] = {
        "min": g_min,
        "max": g_max,
        "nan": g_nan,
        "mean": g_mean,
        "std": g_std,
        "median": g_median
    }

In [236]:
g_stats

{'column_names_correct': True,
 'number_of_rows': 10,
 'patient_id': {'min': 1,
  'max': 10,
  'nan': 0,
  'mean': 5.5,
  'std': 3.0276503540974917,
  'median': [2.6277186767309857, 8.372281323269014]},
 'age': {'min': 33,
  'max': 47,
  'nan': 0,
  'mean': 37.3,
  'std': 5.165053511608353,
  'median': [33, 42.199999999999996]},
 'weight': {'min': 65.9,
  'max': 84.1,
  'nan': 6,
  'mean': 76.575,
  'std': 8.651541288502678,
  'median': [68.36742727720309, 84.1]},
 'hot_encoded': {'min': 0,
  'max': 1,
  'nan': 1,
  'mean': 0.7777777777777778,
  'std': 0.44095855184409843,
  'median': [0.35944776451074, 1]}}